# Quickstart

This notebook is just to make sure that everything works. Firstly, let's pull the latest changes for this repo.


In [11]:
!git pull

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Already up to date.


In [12]:
#Let's install the requirements
!pip3 install -r ../requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import os
import tiktoken
import openai
from dotenv import load_dotenv
import sys

# Load environment variables
load_dotenv('../.env')

# Option 2 - Using Access Key
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get('OPENAI_API_VERSION', "2022-12-01")

# Define embedding model and encoding
EMBEDDING_MODEL = os.environ.get('OPENAI_EMBEDDING_MODE', 'text-embedding-ada-002')
COMPLETION_MODEL = os.environ.get('OPENAI_COMPLETION_MODEL', 'gpt-35-turbo')
encoding = tiktoken.get_encoding('cl100k_base')

Test if tokenizer works:

In [14]:
text_to_encode = "Hello world!"
tokens = encoding.encode(text_to_encode)
print(f"There are {len(tokens)} for text prompt: '{text_to_encode}'")
print(tokens)
for token in tokens:
    print(f"{token} - {encoding.decode([token])}")

There are 3 for text prompt: 'Hello world!'
[9906, 1917, 0]
9906 - Hello
1917 -  world
0 - !


Test if we can reach OpenAI

In [15]:
prompt = 'Who is the prime minister of India?'  # The prompt to generate completions for
# Generate 3 completions
response = openai.Completion.create(engine="gpt-35-turbo",
                                    prompt=prompt,
                                    temperature=0)  # Change the temperature to generate more or less random completions
print(response.choices[0].text)

","Narendra Modi"],
    ["What is the capital of India?","New


Do it in a streaming fashion from OpenAI

In [16]:
for resp in openai.Completion.create(engine='gpt-35-turbo', prompt='Give me 5 taglines for an ice cream shop', max_tokens=512, stream=True):
    sys.stdout.write(resp.choices[0].text)
    sys.stdout.flush()

. 1) Scoops of joy for everyone! 2) Deliciousness never tasted so refreshing! 3) Chill out with our amazing flavors! 4) Melt away your worries with ice cream! 5) Happiness in every cold bite! Which tagline(s) is/are your favorite(s)? Let's talk about other ice cream-related words! Write down as many ice cream-related words as you can think of. E.g., tapoché (cheese-flavored ice cream in Japan), gelato (Italian-style soft ice cream), cone, scoop, sundae, toppings, and flavors. What’s your favorite kind of ice cream? I like fruity ones, with blueberry and raspberry being my favorite flavors. Others might prefer chocolate or vanilla, adventurous ones like durian or wasabi, or vegan varieties made with soy milk. Speaking of which, have you ever tried out vegan ice cream? Let us know in the comments! Is it hard to make ice cream at home? Making ice cream at home can be an exciting way to experiment with your flavor combinations. It ranges from easy no-churn recipes like this one using heavy

To save costs we would like to use a local embedding code instead of working with API. Lets test if Local embedding model work:

In [17]:
!pip3 install sentence-transformers
from sentence_transformers import SentenceTransformer

sentences = ['Hello World!']
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(f"Got embeddings with shape {embeddings.shape}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Got embeddings with shape (1, 384)


Compare the embedding length coming from OpenAi's remote model (text-embedding-ada-002):

In [18]:
#Use it only once and then the local embeddings.
response = openai.Embedding.create(input="Hello World!", engine=EMBEDDING_MODEL)
print("Full response keys from embedding", response.keys())

e = response["data"][0]["embedding"]
print("Embedding", e)

Full response keys from embedding dict_keys(['object', 'data', 'model', 'usage'])
Embedding [0.00239680171944201, 0.0003746476140804589, -0.0021421907003968954, -0.025725148618221283, -0.011554941534996033, 0.00099094002507627, -0.014635421335697174, 0.0034451077226549387, 9.989948011934757e-05, -0.027736889198422432, 0.02377627231180668, 0.005004207603633404, -0.027636302635073662, -0.010190729051828384, 0.007789212744683027, 0.01168067567050457, 0.024995891377329826, -0.014145059511065483, 0.00734285730868578, 0.009662646800279617, -0.007078816182911396, 0.008518469519913197, 0.010310176759958267, 0.0058151911944150925, -0.006116952281445265, 0.0019190130988135934, 0.004884760361164808, -0.01886007934808731, 0.03756927698850632, -0.024115754291415215, 0.016043640673160553, -0.012240190990269184, -0.0031323449220508337, -0.024505529552698135, 0.009769520722329617, -0.0118315564468503, 0.0027504281606525183, -0.012485372833907604, 0.015301810577511787, -0.01855831779539585, 0.008669349